In [7]:
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as sch

#------------------------------------------------------------------------------
#The distance Matrix is define as :
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1
    # This is a proper distance metric
    dist=((1-corr)/2.)**.5 # distance matrix
    return dist

#------------------------------------------------------------------------------
def getQuasiDiag(link):
    # Sort clustered items by distance
    link=link.astype(int)
    sortIx=pd.Series([link[-1,0],link[-1,1]])
    numItems=link[-1,3] # number of original items
    while sortIx.max()>=numItems:
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems
        sortIx[i]=link[j,0] # item 1
        df0=pd.Series(link[j,1],index=i+1)
        sortIx=sortIx.append(df0) # item 2
        sortIx=sortIx.sort_index() # re-sort
        sortIx.index=range(sortIx.shape[0]) # re-index
    return sortIx.tolist()

#------------------------------------------------------------------------------
#Función Recursive Bisection:
def getRecBipart_harp(cov,sortIx,xReturns=None, riskaversion=0.5, minWt = 0.001, maxWt = 1.0):
    # Compute HRP alloc
    w=pd.Series(1,index=sortIx)
    cItems=[sortIx] # initialize all items in one cluster
    while len(cItems)>0:
        cItems=[i[int(j):int(k)] for i in cItems for j,k in ((0,len(i)/2), \
                                                   (len(i)/2,len(i))) if len(i)>1] # bi-section
        for i in range(0,len(cItems),2): # parse in pairs
            cItems0=cItems[i] # cluster 1
            cItems1=cItems[i+1] # cluster 2
            cVar0=getClusterVar(cov,cItems0)
            cVar1=getClusterVar(cov,cItems1)
            if xReturns is None:
                alpha=1-cVar0/(cVar0+cVar1)
                w[cItems0]*=alpha # weight 1 
                w[cItems1]*=1-alpha # weight 2
            else:
                eVar0 = xReturns[cItems0].mean()   # Expected Returns of group 1
                eVar1 = xReturns[cItems1].mean()   # Expected Returns of group 2
                alpha = 1. - cVar0 / (cVar0+cVar1) # HRPi Relative Variance - HRP 'alpha'
                xprtn = 0. + eVar0 / (eVar0+eVar1) # HAPi Expected Return Relative
                
                # High Risk Aversion results in more stock weight from variance vs. returns.
                w[cItems0] *= riskaversion*alpha + (xprtn*(1.-riskaversion))           # HARPi Wt1
                w[cItems1] *= riskaversion*(1.-alpha) + (1.-xprtn)*((1.-riskaversion)) # Wt2                          
        
    return w 

#------------------------------------------------------------------------------
#Función para calcular la varianza por cada cluster:
def getClusterVar(cov,cItems):
    # Compute variance per cluster
    cov_=cov.loc[cItems,cItems] # matrix slice
    w_=getIVP(cov_).reshape(-1,1)
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar

#------------------------------------------------------------------------------
#Función para calcular la inversa de la varianza del portfolio:   
def getIVP(cov,**kargs):
    # Compute the inverse-variance portfolio
    ivp=1./np.diag(cov)
    ivp/=ivp.sum()
    return ivp

#------------------------------------------------------------------------------
def HierarchicalRiskAlphaParity(dataFrame, riskaversion): #DF de la serie de retorno acumulado y el coeficiente de aversión al riesgo
    expReturns = dataFrame.iloc[-1,:]
    expReturns = getxprtnnZeroOne(expReturns)
    corr = dataFrame.corr()
    dist = correlDist(corr)
    link=sch.linkage(dist,method='single', metric='euclidean')
    #Almacenamos los indices ordenados:
    sortIx=getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist() # recover labels
    cov = dataFrame.cov() #Matriz de covarianzas
    hrp=getRecBipart_harp(cov,sortIx,xReturns=expReturns, riskaversion=riskaversion) # Capital allocation
    return hrp


#------------------------------------------------------------------------------
def getxprtnnZeroOne(expReturns=pd.Series):
    # Rescale expected returns from 0 to 1
    if expReturns is None:
        pass
    else:
        idx=expReturns.index
        expReturns = zscore(expReturns)
        expReturns = expReturns/max(abs(expReturns))/2.0+0.5
        expReturns = expReturns / max(abs(expReturns))
        # in case when max(abs()) was -ve, then new max will be < 1.0
        expReturns = pd.Series(expReturns, index=idx)
    return expReturns

def zscore(a, axis=0, ddof=0, keepNaN=False):
    """ NAN Stable Z-Scores"""
    try:
        idx = a.index
    except:
        idx = None
    a = np.asanyarray(a)
    mns = np.nanmean(a, axis=axis)
    sstd = np.nanstd(a=a, axis=axis, ddof=ddof)
    if axis and mns.ndim < a.ndim:
        res = ((a - np.expand_dims(mns, axis=axis)) /
               np.expand_dims(sstd, axis=axis))
    else:
        res = (a - mns) / sstd # RESult
    if not keepNaN:
        res = np.nan_to_num(res) # Default set to zero where was NaN
    res = pd.Series(res, index=idx)
    return res
    